In [1]:
#pip install pandas sqlalchemy psycopg2-binary pyarrow python-dotenv

   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 1.2/1.2 MB 11.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/26.2 MB ? eta -:--:--
   ----------- ---------------------------- 7.3/26.2 MB 34.7 MB/s eta 0:00:01
   ---------------------- ----------------- 14.4/26.2 MB 33.5 MB/s eta 0:00:01
   --------------------------------- ------ 21.8/26.2 MB 33.5 MB/s eta 0:00:01
   ---------------------------------------- 26.2/26.2 MB 31.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import pandas as pd
import os
from datetime import datetime
from dotenv import load_dotenv
from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError


In [4]:
# Cargar variables de entorno
load_dotenv()

# Configuración de PostgreSQL (usando las mismas variables que tu código)
POSTGRES_CONFIG = {
    'dbname': os.getenv('CONSUMER_DB_NAME'),
    'host': os.getenv('CONSUMER_DB_HOST'),
    'port': os.getenv('CONSUMER_DB_PORT', '5432'),
    'user': os.getenv('CONSUMER_DB_USER'),
    'password': os.getenv('CONSUMER_DB_PASSWORD')
}

print("Configuración cargada:")
print(f"Host: {POSTGRES_CONFIG['host']}")
print(f"Database: {POSTGRES_CONFIG['dbname']}")
print(f"User: {POSTGRES_CONFIG['user']}")

Configuración cargada:
Host: strix-consumer-prod-db.cluster-c8qjobbyzxkf.us-east-1.rds.amazonaws.com
Database: analytics
User: strixbox


In [5]:
# Función para conectar a PostgreSQL
def create_postgres_connection():
    try:
        engine = create_engine(
            f"postgresql+psycopg2://{POSTGRES_CONFIG['user']}:{POSTGRES_CONFIG['password']}@"
            f"{POSTGRES_CONFIG['host']}:{POSTGRES_CONFIG['port']}/{POSTGRES_CONFIG['dbname']}"
        )
        print("Conexión a PostgreSQL establecida.")
        return engine
    except SQLAlchemyError as e:
        print(f"Error en conexión a PostgreSQL: {e}")
        return None


In [6]:
# Query para extraer datos de la tabla vehicle
VEHICLE_QUERY = """
SELECT 
    id,
    account_id,
    make,
    year,
    color,
    label,
    model,
    domain,
    subtype,
    engine_number,
    chassis_number,
    mileage,
    lat as latitude,
    long as longitude,
    things,
    location_datetime,
    created_datetime
FROM strix.vehicle
"""

# %%
# Conectar y extraer datos
engine = create_postgres_connection()

if engine:
    print("Ejecutando consulta...")
    df = pd.read_sql(VEHICLE_QUERY, engine)
    print(f"✅ Datos extraídos: {len(df)} registros")
    print(f"Columnas: {list(df.columns)}")
    
    # Mostrar primeras filas
    print("\nPrimeras 5 filas:")
    print(df.head())
else:
    print("No se pudo establecer conexión")


Conexión a PostgreSQL establecida.
Ejecutando consulta...
✅ Datos extraídos: 351060 registros
Columnas: ['id', 'account_id', 'make', 'year', 'color', 'label', 'model', 'domain', 'subtype', 'engine_number', 'chassis_number', 'mileage', 'latitude', 'longitude', 'things', 'location_datetime', 'created_datetime']

Primeras 5 filas:
                                                  id  \
0  mrn:thing:vehicle:58b0101b-ebf5-408c-944d-f7f5...   
1  mrn:thing:vehicle:21ca7177-b754-44e2-934f-a74c...   
2  mrn:thing:vehicle:d13640a5-6aba-4f7d-aa6b-4af2...   
3  mrn:thing:vehicle:c5f0c0d0-9cef-44de-92fd-c5aa...   
4  mrn:thing:vehicle:b4fce3d8-3dbd-468c-a4ca-62dc...   

                                         account_id        make    year  \
0  mrn:account:af34f0d1-197f-42c7-9011-d662c7663aef  VOLKSWAGEN  2013.0   
1  mrn:account:dd3364de-e3af-477a-a311-a5230c74972e     PEUGEOT  2018.0   
2  mrn:account:5fae2f6a-a811-4bc6-a0f0-274da8acc506           X  1234.0   
3  mrn:account:e3ea015c-c64d-4c96

In [6]:
# Generar nombre de archivo con timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f"vehicles_{timestamp}.parquet"
local_path = filename  # Guardar en directorio actual

print(f"Generando archivo: {filename}")

Generando archivo: vehicles_20250725_151356.parquet


In [8]:
# Guardar como parquet
try:
    df.to_parquet(local_path, index=False, engine='pyarrow')
    print(f"✅ Archivo guardado exitosamente: {local_path}")
    print(f"Tamaño del archivo: {os.path.getsize(local_path) / 1024 / 1024:.2f} MB")
except Exception as e:
    print(f"❌ Error guardando archivo: {e}")

❌ Error guardando archivo: name 'df' is not defined


In [9]:
# Verificar el archivo generado
if os.path.exists(local_path):
    # Leer el archivo para verificar
    df_check = pd.read_parquet(local_path)
    print(f"\n📊 Verificación del archivo:")
    print(f"Registros: {len(df_check)}")
    print(f"Columnas: {list(df_check.columns)}")
    print(f"\nTipos de datos:")
    print(df_check.dtypes)
    
    print(f"\n📁 Archivo final: {os.path.abspath(local_path)}")
else:
    print("El archivo no fue creado correctamente")


El archivo no fue creado correctamente


In [10]:
# Información del dataset generado
print("\n" + "="*50)
print("DATASET INFORMACIÓN:")
print("="*50)
print(f"Dataset: vehicles")
print(f"Path: {os.path.abspath(local_path)}")
print(f"File Name Format: vehicles_YYYYMMDD_HHMMSS.parquet")
print(f"Schema:")
print("-" * 30)
for col in df.columns:
    dtype = str(df[col].dtype)
    if 'int' in dtype:
        print(f"{col:<20} INTEGER")
    elif 'float' in dtype:
        print(f"{col:<20} FLOAT")
    elif 'datetime' in dtype:
        print(f"{col:<20} DATETIME")
    else:
        print(f"{col:<20} STRING")


DATASET INFORMACIÓN:
Dataset: vehicles
Path: c:\code\parket\notebook\vehicles_20250725_115305.parquet
File Name Format: vehicles_YYYYMMDD_HHMMSS.parquet
Schema:
------------------------------
id                   STRING
account_id           STRING
make                 STRING
year                 FLOAT
color                STRING
label                STRING
model                STRING
domain               STRING
subtype              STRING
engine_number        STRING
chassis_number       STRING
mileage              FLOAT
latitude             FLOAT
longitude            FLOAT
things               STRING
location_datetime    DATETIME
created_datetime     DATETIME
